In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

import time
import pandas as pd
import numpy as np

# 1. Danawa

In [68]:
# 예제 7-10 상품 정보 태그에서 원하는 정보를 추출하는 함수
def danawa_get_prod_items(prod_items):
    prod_data = []
    for prod_item in prod_items:
        if 'product-pot' in prod_item['class']:
            continue  # 공백부분(‘li.prod_item.product-pot’) 제외하기
        try:
            # ① 상품명 가져오기
            title = prod_item.select('p.prod_name > a')[0].text.strip()
            # ② 스펙 목록 가져오기
            spec_list = prod_item.select('div.spec_list')[0].text.strip()
            # ③ 가격 정보 가져오기
            price = prod_item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",", "")
            # 상품평
            rate = prod_item.select('div.point_num > strong')[0].text.strip()
            # 참여수
            opinion = prod_item.select('a.click_log_prod_content_count > strong')[0].text.strip()
            
            prod_data.append([title, spec_list, price, rate, opinion])
        except:
            pass  # 진행시 에러가 발생할 경우(광고 상품 등) 넘어가기
    return prod_data

def danawa_get_search_page_url(keyword, page):
    return 'http://search.danawa.com/dsearch.php?query={}&volumeType=allvs&page={}&limit=30&sort=saveDESC&list=list&boost=true&addDelivery=N&tab=goods&tab=goods'.format(keyword, page)

def danawa_start_crawling(keyword, total_page):
    # setting
    driver = webdriver.Chrome(service = Service('./chromedriver.exe'))
    driver.implicitly_wait(3)

    prod_data_total = []

    for page in tqdm(range(1, total_page+1)):

        # open chrome browser
        url = danawa_get_search_page_url(keyword, page)
        driver.get(url)
        time.sleep(5)

        # get html source of the page
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # get info
        prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
        prod_item_list = danawa_get_prod_items(prod_items)

        prod_data_total = prod_data_total + prod_item_list
        
        return prod_data_total

In [82]:
prod_data_total = danawa_start_crawling('트리트먼트', 10)

data = pd.DataFrame(prod_data_total)
data.columns = ['title', 'spec_list', 'price', 'rate', 'opinion']
data.to_excel('./treatment_danawa_10page.xlsx', index = False)

  0%|          | 0/10 [00:00<?, ?it/s]

# 2. Naver

In [115]:
# 예제 7-10 상품 정보 태그에서 원하는 정보를 추출하는 함수
def naver_get_prod_items(prod_items):
    prod_data = []
    for prod_item in prod_items:
        if 'product-pot' in prod_item['class']:
            continue  # 공백부분(‘li.prod_item.product-pot’) 제외하기
        try:
            # ① 상품명 가져오기
            title = prod_item.select('a.basicList_link__1MaTN')[0].text.strip()
            # ② 스펙 목록 가져오기
            spec_list = [x.text.strip() for x in prod_item.select('a.basicList_detail__27Krk')]
            # ③ 가격 정보 가져오기
            price = prod_item.select('span.price_num__2WUXn')[0].text.strip()
            # 상품평
            rate = prod_item.select('span.basicList_star__3NkBn')[0].text.split(' ')[-1]
            # 참여수
            opinion = prod_item.select('em.basicList_num__1yXM9')[0].text
            
            prod_data.append([title, spec_list, price, rate, opinion])
        except:
            pass  # 진행시 에러가 발생할 경우(광고 상품 등) 넘어가기
    return prod_data

def naver_get_search_page_url(keyword, page):
    return 'https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery={}&pagingIndex={}&pagingSize=40&productSet=total&query={}&sort=rel&timestamp=&viewType=list'.format(keyword, page, keyword)

def naver_start_crawling(keyword, total_page):
    # setting
    driver = webdriver.Chrome(service = Service('./chromedriver.exe'))
    driver.implicitly_wait(3)

    prod_data_total = []

    for page in tqdm(range(1, total_page+1)):

        # open chrome browser
        url = naver_get_search_page_url(keyword, page)
        driver.get(url)
        time.sleep(1)
        
        # 최하단으로 스크롤링 하여 모든 데이터 표출
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        
        # get html source of the page
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # get info
        prod_items = soup.select('li.basicList_item__2XT81')
        prod_item_list = naver_get_prod_items(prod_items)

        prod_data_total = prod_data_total + prod_item_list
        
    return prod_data_total

In [120]:
prod_data_total = naver_start_crawling('샴푸', 10)

data = pd.DataFrame(prod_data_total)
data.columns = ['title', 'spec_list', 'price', 'rate', 'opinion']
data.to_excel('./shampoo_naver_10page.xlsx', index = False)

  0%|          | 0/10 [00:00<?, ?it/s]

# Coupang

In [1]:
def coupang_get_search_page_url(keyword, page):
    return 'https://www.coupang.com/np/search?q={}&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page={}&rocketAll=false&searchIndexingToken=1=6&backgroundColor='.format(keyword, page)


In [19]:
# setting
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36")

driver = webdriver.Chrome(service = Service('./chromedriver.exe'))
driver.implicitly_wait(3)

keyword = '샴푸'
total_page = 1

prod_data_total = []

for page in tqdm(range(1, total_page+1)):

    # open chrome browser
    url = coupang_get_search_page_url(keyword, page)
    driver.get(url)
    time.sleep(3)

    # get html source of the page
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # get info
    prod_items = soup.select('ul#productList.search-product-list')

  0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
[x.text.strip() for x in soup.select('div.name')]

['메디슈어 탈모 증상 완화 샴푸, 2개, 1L',
 '케라시스 러블리 앤 로맨틱 퍼퓸샴푸 꽃향, 980ml, 3개',
 '미쟝센 퍼펙트 세럼 오리지널 샴푸, 680ml, 4개',
 '쿤달 허니 앤 마카다미아 네이처 샴푸 베이비파우더향, 1058ml, 2개',
 '케라시스 러블리 앤 로맨틱 퍼퓸샴푸 꽃향, 980ml, 3개',
 '닥터포헤어 폴리젠 플러스 샴푸 500ml x 2p + 100ml, 1세트',
 '비오클라쎄 밀크바오밥 샴푸 화이트머스크향, 1000ml, 1개',
 '미쟝센 살롱 플러스 클리닉 10 샴푸 극손상모발용, 990ml, 1개',
 '려 흑운 모근강화 볼륨케어 샴푸, 550ml, 3개',
 '케라시스 데미지 클리닉 샴푸, 750ml, 3개',
 '바론 모링가 리페어링 샴푸 1000g + 헤어 트리트먼트 1000g, 1세트',
 '부케가르니 나드 샴푸 시그니처, 1000ml, 2개',
 'TS 뉴프리미엄 TS 샴푸 500g x 2p + 100g x 2p + 6g x 5p, 1세트',
 '닥터방기원 맥주 효모 탈모증상 완화 샴푸, 1000ml, 2개',
 '닥터방기원 랩 탈모 증상 완화 샴푸, 1L, 2개',
 '제이숲 실크케라틴 단백질 샴푸, 1000g, 1개',
 '트리트룸 시그니처 대용량 퍼퓸 샴푸 화이트머스크향, 1077ml, 2개',
 '부케가르니 나드 두피 딥클렌징 샴푸, 1000ml, 1개',
 '라보에이치 프로바이오틱스 탈모 증상 완화 샴푸 두피강화, 400ml, 1개',
 'TS 뉴프리미엄 탈모 샴푸 라벤더향, 500g, 1개',
 '닥터방기원 댄드러프 비듬케어 탈모샴푸, 1000ml, 1개',
 '려 함빛 극손상 영양케어 샴푸, 550ml, 3개',
 '케라시스 러블리 앤 로맨틱 퍼퓸샴푸 꽃향, 980ml, 3개',
 '닥터포헤어 폴리젠 샴푸, 500ml, 1개',
 '헤드앤숄더 가려운 두피 케어 샴푸, 1200ml, 1개',
 '미쟝센 퍼펙트 세럼 오리지널 샴푸, 680ml, 4개',
 '아모스 녹차실감 지성두피 프레쉬 샴푸

In [31]:
a = [x.text.strip() for x in soup.select('div.name')]

for cnt , i in enumerate(a):
    print(i[:5], end=' ')
    if cnt%4 ==3:
        print()

메디슈어  케라시스  미쟝센 퍼 쿤달 허니 
케라시스  닥터포헤어 비오클라쎄 미쟝센 살 
려 흑운  케라시스  바론 모링 부케가르니 
TS 뉴프 닥터방기원 닥터방기원 제이숲 실 
트리트룸  부케가르니 라보에이치 TS 뉴프 
닥터방기원 려 함빛  케라시스  닥터포헤어 
헤드앤숄더 미쟝센 퍼 아모스 녹 케라시스  
존슨즈베이 케라시스  헤드앤숄더 알페신 카 
부케가르니 닥터방기원 미쟝센 퍼 닥터라파알 
미쟝센 헬 TS 뉴프 헤드앤숄더 헤드앤숄더 
메디슈어  라이프넷  려 흑운  케라시스  
려 청아  아비다선  헤드앤숄더 심플리오  
닥터시드  폴메디슨  씨피원 쓰 부케가르니 
케라시스  미쟝센 살 려 함빛  케라시스  


In [29]:
cnt//4

13